## Asserting Facts into the Engine

You can send Assertions, also known as Facts, into the engine for processing. The engine will match the incoming Assertions against its KB (knowledge base) and trigger any matching rules. Once those rules trigger, they can cause additional rules to trigger, and so on, until no additional assertions are generated.

This tutorial will walk you through the different ways of controlling the assertion process.

Run the following code to get the engine started.

In [1]:
# import the thoughts rules engine
import os, sys
sys.path.insert(1, os.path.abspath('..\\..'))
from thoughts.rules_engine import RulesEngine
from pprint import pprint

engine = RulesEngine()

## A Simple Example - Default Behavior for Assertions

By default when a new fact is asserted, the engine will run continually until no additional assertions are generated. 

It will also return the final conclusions generated by the engine, hiding any interim assertions that were gerneated during processing.

Let's run a trivial example to see both of these behaviors in action.

In [2]:
rules = [
    {"#when": "A", "#then": "B"},
    {"#when": "B", "#then": "C"},
    {"#when": "C", "#then": "D"},
]

engine.load_rules_from_list(rules)

conclusions = engine.process("A")
pprint(conclusions)

['D']


## Viewing the Assertion Tree

Now let's run the same assertion again but this time we'll return the entire set of conclusions, including the interim assertions generated. You can do this by setting the extract_conclusions parameter to False, causing the engine to return the full assertion tree.

Notice how this representation shows how A led to B, which led to C, which led to D. Each conclusion is tracked as sub-conclusion of the parent conclusion which triggered it.

In [4]:
conclusions = engine.process("A", extract_conclusions=False)
pprint(conclusions)

[{'#assert': 'A',
  '#conclusions': [{'#assert': 'B',
                    '#conclusions': [{'#assert': 'C',
                                      '#conclusions': [{'#assert': 'D',
                                                        '#conclusions': []}]}]}],
  '#seq-end': 1,
  '#seq-start': 0}]


## Basic Sequence Rules

In [4]:
rules = [
    
  { "#when": ["A", "B", "C"], "#then": "ABC" },
]

engine.add_rules(rules)

result = engine.process(["A", "B", "D", "A", "B", "C",], extract_conclusions=False)
pprint.pprint(result)

[{'#assert': 'A', '#conclusions': []},
 {'#assert': 'B', '#conclusions': []},
 {'#assert': 'D', '#conclusions': []},
 {'#assert': 'A', '#conclusions': []},
 {'#assert': 'B', '#conclusions': []},
 {'#assert': 'C', '#conclusions': [{'#assert': 'ABC', '#conclusions': []}]}]


In [ ]:
rules = [
    
  { "#when": "cloudy", "#then": "rain possible" },
  { "#when": "cold", "#then": "snow possible" },

  {
    "#when": "predict weather when ?condition",
    "#then": [
      { "#assert": "?condition", "#into": "$prediction" },
    ]
  }
]

engine.add_rules(rules)

engine.process("predict weather when cloudy")
pprint.pprint(engine.context.items)

engine.process("predict weather when cold")
pprint.pprint(engine.context.items)

In [ ]:
rule = {
    "#when": "it's ?condition today",
    "#then": "predict weather when ?condition"
}

engine.add_rule(rule)

engine.process("it's cold today")
pprint.pprint(engine.context.items)